# Menedżery kontekstu

Od podstaw `with` po zaawansowane konstrukcje i własne implementacje.


## Cele
- przypomnieć działanie protokołu kontekstowego (`__enter__` / `__exit__`)
- stworzyć własny menedżer przy użyciu klasy i dekoratora `contextmanager`
- wykorzystać konteksty do obsługi zasobów i transakcji


## Podstawy `with`
Menedżer kontekstu zapewnia deterministyczne sprzątanie zasobów.
Najczęściej spotykamy go przy pracy z plikami.


In [ ]:
from pathlib import Path

path = Path("notes.txt")

with path.open("w", encoding="utf-8") as handle:
    handle.write("Linia 1
")

print(path.read_text(encoding="utf-8"))


## Własny menedżer w formie klasy
Implementacja wymaga metod `__enter__` oraz `__exit__`.


In [ ]:
class suppress:
    def __init__(self, *exceptions):
        self.exceptions = exceptions

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc, tb):
        if exc_type and issubclass(exc_type, self.exceptions):
            print("Wyjątek został stłumiony:", exc)
            return True  # sygnalizujemy, że wyjątek obsłużony
        return False

with suppress(ZeroDivisionError):
    1 / 0


## Dekorator `contextmanager`
Moduł `contextlib` pozwala definiować menedżery jako generatory.


In [ ]:
from contextlib import contextmanager

@contextmanager
def temporary_directory(path: Path):
    path.mkdir(exist_ok=False)
    try:
        yield path
    finally:
        for child in path.iterdir():
            child.unlink()
        path.rmdir()

with temporary_directory(Path("sandbox")) as tmp:
    file_path = tmp / "data.txt"
    file_path.write_text("próba", encoding="utf-8")
    print(file_path.read_text(encoding="utf-8"))
print("Czy katalog istnieje?", Path("sandbox").exists())


**Podsumowanie:** Menedżery kontekstu standaryzują zarządzanie zasobami.

**Pytanie kontrolne:** Kiedy warto użyć `contextmanager`, a kiedy klasy?


### 🧩 Zadanie 1
Zaimplementuj menedżer kontekstu, który mierzy czas wykonania bloku i raportuje wynik.


In [ ]:
# Rozwiązanie Zadania 1
import time
from contextlib import contextmanager

@contextmanager
def timer(name: str):
    start = time.perf_counter()
    try:
        yield
    finally:
        elapsed = (time.perf_counter() - start) * 1000
        print(f"[{name}] {elapsed:.2f} ms")

with timer("operacja ciężka"):
    sum(range(1_000_000))


### 🧩 Zadanie 2
Stwórz menedżer `transaction`, który pracując na słowniku symuluje transakcję:
zmiany są widoczne w bloku, a po wyjściu albo się zatwierdzają (`commit=True`),
albo wycofują.


In [ ]:
# Rozwiązanie Zadania 2
class transaction:
    def __init__(self, storage: dict, commit: bool):
        self.storage = storage
        self.commit = commit

    def __enter__(self):
        self._snapshot = self.storage.copy()
        return self.storage

    def __exit__(self, exc_type, exc, tb):
        if exc_type or not self.commit:
            self.storage.clear()
            self.storage.update(self._snapshot)
        # brak zwrotu True -> wyjątek ewentualnie propaguje dalej

store = {"balance": 100}
with transaction(store, commit=True) as data:
    data["balance"] += 50
print(store)

with transaction(store, commit=False) as data:
    data["balance"] += 999
print(store)
